# 0.0. Imports

In [1]:
import pickle
import inflection

import numpy as np
import pandas as pd
import seaborn as sn

from xgboost import XGBClassifier
from ydata_profiling import ProfileReport

from sklearn import metrics as mt
from sklearn import ensemble as en
from sklearn import model_selection as ms
from sklearn import preprocessing   as pp

from sklearn import linear_model as lm
from sklearn import ensemble as en


/home/felipe/repos/hackday/hackday/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1.0. Data Description

## 1.1. Loading Data

In [2]:
# load datasets
df_raw = pd.read_csv('data/test.csv')
df_country = pd.read_csv('data/country_base_2.csv')

# mergre county coordinates
df1 = pd.merge(df_raw, df_country, on='Nacionalidade', how='left')

# drop irrelevant columns
df1 = df1.drop('country', axis=1)

## 1.2. Rename Columns

In [3]:
# list columns names
cols_old = ['id', 'Classificacao_do_hotel', 'Meses_da_reserva_ate_o_check-in', 'Numero_de_pernoites_reservadas', 
            'Numero_de_hospedes', 'Regime_de_alimentacao', 'Nacionalidade', 'Forma_de_Reserva',
            'Ja_se_hospedou_anterioremente', 'Tipo_do_quarto_reservado', 'Reserva_feita_por_agencia_de_turismo', 
            'Reserva_feita_por_empresa', 'Reserva_com_Estacionamento', 'Reserva_com_Observacoes', 'latitude', 'longitude']

# rename columns
snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))
df1.columns = cols_new

### 1.4.1. Replace NA

In [4]:
# replace with mode
df1['numero_de_hospedes'] = df1['numero_de_hospedes'].fillna(2)
df1['nacionalidade'] = df1['nacionalidade'].fillna('Spain')
df1['latitude'] = df1['latitude'].fillna(40463667)
df1['longitude'] = df1['longitude'].fillna(-374922)

### 1.5.1. Change Dtypes

In [5]:
df1['numero_de_hospedes'] = df1['numero_de_hospedes'].astype('int')

# 2.0. Feature Engineering

In [6]:
df2 = df1#.copy()

In [7]:
# classificacao_do_hotel
df2['classificacao_do_hotel'] = df2['classificacao_do_hotel'].apply(lambda x: x.replace(' estrelas', ''))

# ja_se_hospedou_anterioremente
df2['ja_se_hospedou_anterioremente'] = df2['ja_se_hospedou_anterioremente'].apply(lambda x: x.replace('Sim', '1'))
df2['ja_se_hospedou_anterioremente'] = df2['ja_se_hospedou_anterioremente'].apply(lambda x: x.replace('Não', '0'))

# reserva_feita_por_agencia_de_turismo
df2['reserva_feita_por_agencia_de_turismo'] = df2['reserva_feita_por_agencia_de_turismo'].apply(lambda x: x.replace('Sim', '1'))
df2['reserva_feita_por_agencia_de_turismo'] = df2['reserva_feita_por_agencia_de_turismo'].apply(lambda x: x.replace('Não', '0'))

# reserva_feita_por_empresa
df2['reserva_feita_por_empresa'] = df2['reserva_feita_por_empresa'].apply(lambda x: x.replace('Sim', '1'))
df2['reserva_feita_por_empresa'] = df2['reserva_feita_por_empresa'].apply(lambda x: x.replace('Não', '0'))

# reserva_com_estacionamento
df2['reserva_com_estacionamento'] = df2['reserva_com_estacionamento'].apply(lambda x: x.replace('Sim', '1'))
df2['reserva_com_estacionamento'] = df2['reserva_com_estacionamento'].apply(lambda x: x.replace('Não', '0'))

In [8]:
# transform into int
df2['classificacao_do_hotel'] = df2['classificacao_do_hotel'].astype('int')
df2['ja_se_hospedou_anterioremente'] = df2['ja_se_hospedou_anterioremente'].astype('int')
df2['reserva_feita_por_agencia_de_turismo'] = df2['reserva_feita_por_agencia_de_turismo'].astype('int')
df2['reserva_feita_por_empresa'] = df2['reserva_feita_por_empresa'].astype('int')
df2['reserva_com_estacionamento'] = df2['reserva_com_estacionamento'].astype('int')

# 3.0. Data Filtering

In [9]:
df3 = df2#.copy()

In [10]:
# reserva_feita_por_agencia_de_turismo reserva_feita_por_empresa
# df3 = df3.drop(['reserva_feita_por_agencia_de_turismo', 'reserva_feita_por_empresa'], axis=1)

# 4.0. Data Preparation 

In [11]:
df4 = df3#.copy()

In [12]:
## frequency encoder
#tipo_do_quarto_reservado
freq_dict = df4['tipo_do_quarto_reservado'].value_counts(normalize=True).to_dict()
df4['tipo_do_quarto_reservado_encoded'] = df4['tipo_do_quarto_reservado'].map(freq_dict)

# nacionalidade
freq_dict = df4['nacionalidade'].value_counts(normalize=True).to_dict()
df4['nacionalidade'] = df4['nacionalidade'].map(freq_dict)

In [13]:
# dummy variables
cols_dummy = ['regime_de_alimentacao', 'forma_de_reserva', 'reserva_com_observacoes']
df4_dummy = pd.get_dummies(df4[cols_dummy])
df4 = pd.concat([df4, df4_dummy], axis=1)

In [14]:
# transform lat long into radians
df4['latitude'] = df4['latitude'].apply(lambda x: np.radians(x/1000000))
df4['longitude'] = df4['longitude'].apply(lambda x: np.radians(x/1000000))

# 5.0. Feature Selection 

In [15]:
df5 = df4#.copy()

In [16]:
# drop columns
cols_drop =['regime_de_alimentacao', 'forma_de_reserva', 'reserva_com_observacoes', 'tipo_do_quarto_reservado']
df5 = df4.drop(cols_drop, axis=1)

# 6.0. Machine Learning Model

## 6.1. XGBoost

In [17]:
# load trained xgb_model
xgb_model = pickle.load(open('xgb_model.pkl', 'rb'))

# predict
yhat_xgb = xgb_model.predict(df5)

## 6.1. Gradient Boost

In [18]:
# load gradient boosting model
gdr_model = pickle.load(open('gdr_model.pkl', 'rb'))

# predict
yhat_gb = gdr_model.predict(df5)

## 6.1. Random Forest

In [19]:
# load random forest model
rf_model = pickle.load(open('rf_model.pkl', 'rb'))

# predict
yhat_rf = rf_model.predict(df5)

## 6.1. Ensemble

In [20]:
# get models predicion and create a dataframe
ensemble = pd.DataFrame([yhat_xgb, yhat_gb, yhat_rf]).T

# rename columns
ensemble.columns = ['yhat_xgb', 'yhat_gb', 'yhat_rf']

# voting system - find mode and assing it to a new column
moda = ensemble.mode(axis=1)[0]
ensemble['pred'] = moda

# get final prediction
yhat_en = ensemble['pred']

# 7.0. Export CSV

In [21]:
# add prediciont to original dataset
df2['Reserva Cancelada'] = yhat_en

# select columns required in kaggle competition
submission = df2[['id', 'Reserva Cancelada']].copy()

# export csv
submission.to_csv('submission.csv', index=False)